In [1]:
from lib.reader import read_alleles_to_num
import numpy as np
import os

from sklearn.model_selection import LeaveOneOut

import argparse

from lib.config import *
import lib.row_cheker as rch

import time

In [2]:
import multiprocessing
import pickle

In [3]:
from importlib import reload

In [4]:
rch = reload(rch)

In [5]:
np.random.seed(239)

parser = argparse.ArgumentParser()
parser.add_argument('-i', type=str, dest='input_table_path')
parser.add_argument('-o', type=str, dest='output_table_path')
parser.add_argument('-n', type=int, dest='rows_number', default=500)
args = parser.parse_args(["-i", "../data_as_npy.npy", "-o", "../res"])

In [6]:
args

Namespace(input_table_path='../data_as_npy.npy', output_table_path='../res', rows_number=500)

In [7]:
table = np.load(args.input_table_path).astype(int)
print(table.shape)


(428, 4382202)


In [8]:
#case_control = np.array([0] * (407-21) + [1] * 21)
#np.random.shuffle(case_control)
case_control = np.load("../1_stage_random_split/case_control.npy")
print(case_control.shape, case_control.sum())

(407,) 21


In [9]:
case_control.sum()

21

In [10]:
table = table[:407]
table.shape

(407, 4382202)

In [11]:
print("Starting on original table...")
#table_2x3 = rch.check_initial_table(table, case_control.astype(bool))
table_2x3 = np.load("../1_stage_random_split/table_2x3.npy")
assert all(table_2x3.sum(1).sum(1) == 407)

Starting on original table...


In [13]:
np.save("../1_stage_random_split/table_2x3", table_2x3)

In [14]:
np.save("../1_stage_random_split/case_control", case_control)

In [12]:
table_2x3.shape

(4382202, 2, 3)

In [13]:
loo = LeaveOneOut()

In [14]:
target_rows_list = []
target_pvals_list = []

chi2_hash = pickle.load(open("../1_stage_random_split/chi2_hash_3",  'rb'))
#chi2_hash = dict()

In [15]:
len(chi2_hash)

296721

In [16]:
def do_all_for_single_split(index, table_2x3, case_control, table, chi2_hash, row_number, upd_hash=False):
    train_index, test_index = index[0], index[1]
    beg = time.time()
    rows, pvals = rch.check_loo_table(table_2x3,
                                      case_control[test_index],
                                      table[test_index].flatten(),
                                      chi2_hash,
                                      upd_hash,                                  
                                      cut=args.rows_number)
    end = time.time()
    #print("Done with index {} in {} seconds".format(test_index, end - beg))
    
    return rows, pvals, end - beg

In [17]:
train_test_indexes = [[x,y] for x,y in loo.split(table)]

In [27]:
max([x[1][0] for x in train_test_indexes])

406

In [31]:
for i in range(1,3):
    rows, pvals, time_seconds = do_all_for_single_split(train_test_indexes[i],
                                                table_2x3,
                                                case_control,
                                                table,
                                                chi2_hash,
                                                args.rows_number,
                                                True)
    target_rows_list.append(rows)
    target_pvals_list.append(pvals)
    print("Done with {} in {}".format(i, time_seconds))

  0%|          | 188/4382202 [00:00<38:56, 1875.78it/s]

Done with 1 in 1049.2260670661926


100%|██████████| 4382202/4382202 [17:14<00:00, 4236.06it/s]


Done with 2 in 1035.2143023014069


In [33]:
print("Done with {} in {}".format(i, time_seconds))

Done with 2 in 1035.2143023014069


In [29]:
len(chi2_hash)

296721

In [38]:
pickle.dump(chi2_hash, open("../1_stage_random_split/chi2_hash_3", 'wb'))

In [39]:
target_rows_list

[array([ 285179, 4102717,  285183,  285174, 3997273,    2314, 4248907,
        1656078,    2320,   30232,   30241, 3017932, 3017912, 2561183,
        1533741, 1533653,  895351, 1552508, 1533738, 2784006, 4016660,
         719358, 3018484, 3018470, 2784105,  285180, 2494871, 3489256,
        3489258, 3489242, 3489244, 3489235, 3052111, 3622033,  476040,
        3652051, 1968724, 2432372,  476052, 2384146, 1835495, 1505997,
        1343674, 4365086, 4181738, 1994165, 4016864,  607771, 3052106,
        3052113,  285176,  362405,  362466, 4178141,  902464, 4178121,
        2576342,  225965, 2770837, 2770836,  225945, 4335231, 3688708,
        1524189, 3688884, 1509375,  582406, 3688761, 3622034, 2537584,
        2782808, 2005487, 2005488, 2005505, 2741323, 2005501, 3018088,
        3688971, 3584609, 3622013, 3622030, 3622028, 3622008, 3622027,
         362640,  362634,  362614,  362608,  362585,  362637,  362628,
        3688975, 3584613, 2882028, 1524181, 1524191, 2494872, 2832583,
      

In [40]:
all_rows = set()
for rows in target_rows_list:
    for row in rows:
        all_rows.add(row)

In [41]:
len(all_rows)

505

## test MP

In [19]:
from multiprocessing import pool

In [23]:
pool = multiprocessing.Pool(32)

100%|██████████| 4382202/4382202 [1:49:49<00:00, 665.07it/s]


In [21]:
def g(x):
    return do_all_for_single_split(x, table_2x3, case_control, table, chi2_hash, args.rows_number)

In [22]:
pool.close()

In [ ]:
answ_32_ = pool.map(g, train_test_indexes[32:])

In [30]:
len(answ_32_)

375

In [ ]:
answ_64_ = pool.map(g, train_test_indexes[64:])

In [29]:
print("322")

322


In [20]:
[x[1] for x in train_test_indexes]

[array([0]),
 array([1]),
 array([2]),
 array([3]),
 array([4]),
 array([5]),
 array([6]),
 array([7]),
 array([8]),
 array([9]),
 array([10]),
 array([11]),
 array([12]),
 array([13]),
 array([14]),
 array([15]),
 array([16]),
 array([17]),
 array([18]),
 array([19]),
 array([20]),
 array([21]),
 array([22]),
 array([23]),
 array([24]),
 array([25]),
 array([26]),
 array([27]),
 array([28]),
 array([29]),
 array([30]),
 array([31]),
 array([32]),
 array([33]),
 array([34]),
 array([35]),
 array([36]),
 array([37]),
 array([38]),
 array([39]),
 array([40]),
 array([41]),
 array([42]),
 array([43]),
 array([44]),
 array([45]),
 array([46]),
 array([47]),
 array([48]),
 array([49]),
 array([50]),
 array([51]),
 array([52]),
 array([53]),
 array([54]),
 array([55]),
 array([56]),
 array([57]),
 array([58]),
 array([59]),
 array([60]),
 array([61]),
 array([62]),
 array([63]),
 array([64]),
 array([65]),
 array([66]),
 array([67]),
 array([68]),
 array([69]),
 array([70]),
 array([71]),
 a

In [31]:
for i, (r, p, t) in enumerate(answ_32_):
    print(i+32, t)
    np.save("../1_stage_random_split/rows/rows_" + str(i+32), r)
    np.save("../1_stage_random_split/pvals/pvals_" + str(i+32), p)

32 2255.945820093155
33 2501.088585138321
34 2385.5412735939026
35 2268.300317764282
36 2345.0908586978912
37 2236.6212515830994
38 2261.593354701996
39 2320.035749912262
40 2404.151049375534
41 14604.910432815552
42 1985.3738062381744
43 1936.8774433135986
44 14635.630736351013
45 2038.5704410076141
46 1913.0534329414368
47 2485.2557849884033
48 2506.0797221660614
49 2401.1403501033783
50 2287.2372410297394
51 2506.903571844101
52 2416.24738240242
53 14642.462574958801
54 1984.5760657787323
55 1933.5961649417877
56 2282.995105743408
57 2518.8710350990295
58 2257.4763753414154
59 14147.699615240097
60 1959.2111475467682
61 1891.7407140731812
62 2333.911036014557
63 2236.0599884986877
64 2153.0934648513794
65 2482.7897930145264
66 2500.9565200805664
67 2462.834121465683
68 2443.8523666858673
69 2454.125769138336
70 2367.68842625618
71 2458.877447605133
72 2498.4945838451385
73 2412.4480106830597
74 2441.57390999794
75 2376.10777926445
76 2187.0150146484375
77 2450.167333126068
78 2274.4

In [157]:
all(answ_0_10[2][0] == target_rows_list[3])

True

In [143]:
target_rows_list[2] == target_rows_list[3]

array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True,  True, False, False,  True, False, False, False,
       False, False, False,  True, False, False, False, False,  True,
        True,  True,  True, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True, False,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False,  True,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [ ]:
print("322")

# COLLECT GOOD ROWS

In [5]:
table = np.load(args.input_table_path).astype(int)
print(table.shape)
case_control = np.array([0] * 407 + [1] * 21)

(428, 4382202)


In [6]:
rows_path = "../1_stage_results/rows/"
rows_files = os.listdir(rows_path)

In [7]:
all_rows = []
for f_name in rows_files:
    all_rows.append(np.load(rows_path+f_name))

In [10]:
len(all_rows[0])

500

In [12]:
rows_to_save = set()
for ind in all_rows:
    for row in ind:
        rows_to_save.add(row)

In [20]:
mask = np.array([False] * table.shape[1])
mask[np.array(list(rows_to_save))] = True
print(mask.shape, mask.sum())

(4382202,) 1406


In [24]:
selected_rows = table[:, mask]

In [26]:
head = np.arange(mask.shape[0])[mask]

In [32]:
head

array([  10288,   10612,   16126, ..., 4359974, 4361694, 4366824])

In [34]:
import pandas as pd

In [36]:
result = pd.DataFrame(selected_rows, columns=head)

In [40]:
all(result[10612] == table[:, 10612])

True

In [43]:
for c in result.columns:
    if not all(result[c] == table[:,c]):
        print(c)

In [45]:
result.to_csv("../1_stage_results/result.csv")

In [51]:
np.where(np.array(rows_files) == "rows_0.npy")

(array([217]),)

In [52]:
np.load("../1_stage_results/rows/rows_0.npy") == all_rows[217]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,